In [246]:
!pip install netCDF4 
!pip install matplotlib 
!pip install numpy 
!pip install folium
!pip install geojsoncontour
!pip install branca
!pip install selenium
!pip install imageio

     |████████████████████████████████| 3.3 MB 3.9 MB/s eta 0:00:01


In [4]:
import folium 
from wrf import getvar, interplevel, to_np, latlon_coords
from folium import plugins
from netCDF4 import Dataset as NetCDFFile
import matplotlib.pyplot as plt
import numpy as np
import geojsoncontour
import json
import branca.colormap as cm
from matplotlib import colors as mcolors
import selenium.webdriver
import os
import time
import imageio
from PIL import Image
from pathlib import Path
from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = selenium.webdriver.Firefox(executable_path='/usr/bin/geckodriver',options=options)

In [14]:
nc = NetCDFFile('/work/syseng/wrfout_d01_2020-01-01_00:00:00')
date=getvar(nc,'Times')
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V4.2 MODEL
    START_DATE: 2020-01-01_00:00:00
    SIMULATION_START_DATE: 2020-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION: 100
    SOUTH-NORTH_GRID_DIMENSION: 100
    BOTTOM-TOP_GRID_DIMENSION: 33
    DX: 1000.0
    DY: 1000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 6
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 91
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 1
    CU_PHYSICS: 16
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 3
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0


In [12]:
def get_data(data: NetCDFFile, t: int, z: int):
    z *= 1000
    height = getvar(data, "height", timeidx=t)
    u_all = getvar(data, 'ua', timeidx=t)
    v_all = getvar(data, 'va', timeidx=t)
    w_all = getvar(data, 'wa', timeidx=t)
    T_all=getvar(nc,'tc')
    P_all=getvar(nc,'pressure')
    P = interplevel(P_all, height, z)
    T = interplevel(T_all, height, z)
    u = interplevel(u_all, height, z)
    v = interplevel(v_all, height, z)
    w = interplevel(w_all, height, z)
    return T, u, v, np.sqrt(u**2+v**2+w**2), P
    
def get_folium(i,op):
    T, U, V, mag, P= get_data(nc, t=i, z=1)
    lats, lons = latlon_coords(mag)
    figure = plt.figure()
    ax = figure.add_subplot(111)
    
    if op=='Wind':
        capt="Wind Velocity in m/s"
        var=mag
    elif op=='Temp':
        capt="Tempeture in Celsius"
        var=T
    elif op=='U_Wind':
        capt="U_Wind Velocity in m/s"
        var=U
    elif op=='V_Wind':
        capt="V_Wind Velocity in m/s"
        var=V
    elif op=='Pressure':
        capt="Pressure in hPa"
        var=P
    else:
        print('Debe ingresar opción valida (mag, temp, u , v)')
        return null
    
    contour = ax.contourf(lons, lats, var, cmap=plt.cm.jet)
    cbar = figure.colorbar(contour)
    
    gj_str = geojsoncontour.contourf_to_geojson(
        contourf=contour,
        ndigits=3,
          unit='m'
    )
    gj=json.loads(gj_str)


    folium_map = folium.Map(location=[lats.mean()-0.08, lons.mean()], tiles="Cartodb Positron",
                      zoom_start=11,zoom_control=False,scrollWheelZoom=False,dragging=False)

    folium.GeoJson(gj, style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.2,
        },name="geojson").add_to(folium_map)

    colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'green','greenyellow','yellow', 'orange', 'red','darkred'],
                                 index=np.array(cbar.values), vmin=cbar.values[0], vmax=cbar.values[len(cbar.values)-1],
                                 caption=capt)
    folium_map.add_child(colormap)
    return folium_map,figure

def get_image(val):
    i , op= val
    path=os.getcwd()+'/'
    f_map, fig=get_folium(i,op)
    name=op+'_BQ'+str(i)
    f_map.save(name+'.html')
    driver.set_window_size(800, 800) 
    driver.get('file://'+path+name+'.html')
    time.sleep(3)
    driver.save_screenshot(name+'.png')
    img=imageio.imread(name+'.png')
    os.remove(name+'.html')
    os.remove(name+'.png')
    return img

In [10]:
op='Pressure'
horas=42
results=[]
for i in range(horas):
    results.append(get_image((i,op)))

imageio.mimwrite(op+'_GifMap'+str(date.values)+'.gif', results, fps=2)

<ipython-input-8-2a8680cbf4fe>:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure = plt.figure()
